<a href="https://colab.research.google.com/github/nahaerin/Project_stock/blob/main/Data_Crawling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import re                     
import requests               
from bs4 import BeautifulSoup 
import pandas as pd 

In [172]:
##### 코스피 200 종목 코드 가져오는 크롤링 코드
res = {'code' : [], 'name' : []}    # 데이터 저장하려고 만든 것

for pages in range(1,21):   # 총 20페이지였음
    url = f"https://finance.naver.com/sise/entryJongmok.nhn?&page={pages}"
    req = requests.get(url, headers={'User-agent':'Mozilla/5.0'})
    html = BeautifulSoup(req.text, 'lxml')
    items = html.find_all('td', {'class':'ctg'})            # html -> <td class="ctg"><a href="/item/main.nhn?code=005930" target="_parent">삼성전자</a></td>

    for item in items:
        txt = item.a.get('href')                     # href를 가져오게 되면 "/item/main.nhn?code=005930"에서 뒤에 종목코드만 빼오면 됨.   
        cc = re.search('[\d]+', txt)                 # 정규표현식 사용. [\d] 숫자표현, + : 반복
        
        res['code'].insert(0, cc.group())            # 종목코드 저장
        res['name'].insert(0, item.text)             # 종목명 저장

kospi200 = pd.DataFrame(data=res)     # 데이터프레임 형식으로 저장
kospi200.head()

,code,name
0,009410,태영건설
1,145990,삼양사
2,005180,빙그레
3,005610,SPC삼립
4,001060,JW중외제약


In [171]:
# csv 파일로 저장
kospi200.to_csv('/content/drive/MyDrive/Multicampus/Python/Project/data/kospi200_code.csv', index=False)

In [117]:
##### 주식 종목 코드 입력하면 일별시세 데이터 가져오는 크롤링 코드
def stock_crawling(code, page_num):
    
    res = {'index': [], 'data': []}         # 데이터 저장하려고 만든 것

    for pages in range(page_num):
        url = f'http://finance.naver.com/item/sise_day.nhn?code={code}&page={pages}'   # 가져올 웹페이지 주소
        req = requests.get(url, headers={'User-agent':'Mozilla/5.0'})                  # response 해주기
        html = BeautifulSoup(req.text, 'lxml')                                         # html 가져오기
        items = html.find_all("tr", {"onmouseover":"mouseOver(this)"})                # 데이터 있는 부분 

        for item in items:
            tdList = item.find_all('td')
            date = tdList[0].text.strip()                               # 날짜
            closePrice = int(tdList[1].text.strip().replace(',', ''))   # 종가
            diffPrice = int(tdList[2].text.strip().replace(',', ''))    # 전일비
            openPrice = int(tdList[3].text.strip().replace(',', ''))    # 시가
            highPrice = int(tdList[4].text.strip().replace(',', ''))    # 고가
            lowPrice = int(tdList[5].text.strip().replace(',', ''))     # 저가
            volume = int(tdList[6].text.strip().replace(',', ''))       # 거래량
            
            res['index'].insert(0, date)                                                 # 인덱스를 날짜로
            res['data'].insert(0, [highPrice, lowPrice, openPrice, closePrice, volume])  # 고가, 저가, 시가, 종가, 거래량만 데이터로 저장

    return pd.DataFrame(data=res['data'], index = res['index'], columns=["고가","저가","시가","종가","거래량"])   # 데이터프레임 형식으로 반환

In [174]:
# kospi200.code 별로 주식 데이터 가져온거 아래로 붙이기..? 